In [4]:
import os
def set_api_key(filepath: str):
    with open(filepath, 'r') as f:
        for line in filepath:
            if line.startswith("OPENAI_API_KEY"):
                print(line.strip().split("="))
        # os.environ['OPENAI_API_KEY'] = 

In [ ]:
def read_system_prompt():
    pass

In [12]:
from openai import OpenAI

client = OpenAI()

tools = [
  {
      "type": "function",
      "function": {
          "name": "get_weather",
          "parameters": {
              "type": "object",
              "properties": {
                  "location": {"type": "string"}
              },
          },
      },
  }
]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "What is the weather like in Austin, TX today"
        }
    ],
    tools=tools
)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_nJ2xwYVLZyTRujFSZZx5S1fS', function=Function(arguments='{"location":"Austin, TX"}', name='get_weather'), type='function')]
